# Final: saving all HMM data, of which the states timecourses.

## The libraries and methods we need:

In [1]:
import numpy as np
from loader import load_oneIC

In [2]:
from sklearn.decomposition import PCA
from hmmlearn import hmm
import scipy.signal as signal
import pickle

In [3]:
import matplotlib.pyplot as plt

In [4]:
import h5py

In [5]:
import xarray as xr

In [6]:
directory = "E:/timot/Documents/1 - Centrale Marseille/0.5 - Semestre S8/Stage/NIC_250819"
file = "FCK_LOCKED_IC_JYOTIKA_250819.mat"

path = f"{directory}/{file}"

mat_file = h5py.File(path, "r")
cells_refs = mat_file['FCK_LOCKED_IC_JYOTIKA']

n_IC = 4
n_subj = 23

## The routine:

In [7]:
# The parameters we change to hope for some results
lags = np.arange(-29, 29)
n_lags = lags.shape[0]
n_iter=100
n_states=3    # for the Hidden Markov Model
n_components=40     # For the principal component analysis
covariance_type='diag'
model_type='GMMHMM'
tol=0.01
n_mix=1

origin_dir = "tde-hmm2/19.06 nc_files/"
target_dir = "tde-hmm2/nc_files/"

In [86]:
subj_list = [i for i in range(2,9)] + [i for i in range(10,13)] + [14] + [i for i in range(16, n_subj+1)] 
                                                                            # All except subjects 1, 9, 13, 15

for subj in subj_list:
    tcourses = []
    psds = []
    IC_list = []
    for IC in range(1, n_IC+1):
        try:
            print(f"---- SUBJECT{subj}, IC{IC} ----")

            data, n_trials = load_oneIC(mat_file, cells_refs, subj, IC, comp=False)
                        
            tcourse_disk = xr.open_dataset(f"tde-hmm2/19.06 nc_files/su{subj}IC{IC}-states_timecourse_256Hz.nc")
            tcourse = np.concatenate(
                (np.zeros((29,n_states)), tcourse_disk["states_timecourse_256Hz"].values, np.zeros((28,n_states)))
            )
            
            psddata = xr.open_dataset(f"tde-hmm2/19.06 nc_files/su{subj}IC{IC}-states_psd.nc")
            psds.append(
                psddata["states_psd"].values[np.newaxis]
            )
            
            t_len = data["time_axis"].shape[0]
            tcourse_trials = np.zeros((1, n_trials, t_len, n_states))
            for tr in range(n_trials):
                tcourse_trials[0, tr] = tcourse[tr*t_len:(tr+1)*t_len] 
            tcourses.append(tcourse_trials)
            IC_list.append(IC)
        except:
            print(f"subj{subj}, IC{IC}: NOT POSSIBLE")
    ds = xr.Dataset(
        {
            "states_timecourse": (("IC","trials","time", "states"), np.concatenate((tcourses))),
            "states_psd": (("IC", "states", "freq"), np.concatenate((psds))),
        },
        {
            "IC":IC_list,
            "time":data["time_axis"],
            "states":np.arange(n_states),
            "freq": freqs,
        }
    )
    ds.to_netcdf(f"tde-hmm2/nc_files/su{subj}-{n_states}states_data.nc")

---- SUBJECT3, IC1 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT3, IC2 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT3, IC3 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT3, IC4 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT4, IC1 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT4, IC2 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT4, IC3 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT4, IC4 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet tran

---- SUBJECT21, IC4 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT22, IC1 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT22, IC2 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT22, IC3 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT22, IC4 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT23, IC1 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT23, IC2 ----
Loading the raw timecourse
Computing and loading the time-frequency wavelet transformation for 3 trials
---- SUBJECT23, IC3 ----
Loading the raw timecourse
The independent component IC3 of the subject 

In [89]:
for subj in subj_list:
    ds = xr.open_dataset(f"tde-hmm2/nc_files/su{subj}-3states_data.nc")
    print(f"----SUBJ{subj}----")
    print(ds.keys)

----SUBJ3----
<bound method Mapping.keys of <xarray.Dataset>
Dimensions:            (IC: 4, freq: 196, states: 3, time: 1793, trials: 673)
Coordinates:
  * time               (time) float64 -4.0 -3.996 -3.992 ... 2.992 2.996 3.0
  * freq               (freq) float64 0.0 0.2551 0.5102 ... 49.23 49.49 49.74
  * IC                 (IC) int32 1 2 3 4
  * states             (states) int32 1 2 3
Dimensions without coordinates: trials
Data variables:
    states_timecourse  (IC, trials, time, states) float64 ...
    states_psd         (IC, states, freq) float64 ...>
----SUBJ4----
<bound method Mapping.keys of <xarray.Dataset>
Dimensions:            (IC: 4, freq: 196, states: 3, time: 1793, trials: 760)
Coordinates:
  * time               (time) float64 -4.0 -3.996 -3.992 ... 2.992 2.996 3.0
  * freq               (freq) float64 0.0 0.2551 0.5102 ... 49.23 49.49 49.74
  * IC                 (IC) int32 1 2 3 4
  * states             (states) int32 1 2 3
Dimensions without coordinates: trials
Dat

We want to access the data in our .nc files and then use analyse them to have plots etc.

For example, we could want to :
- Plot the states timecourse over a tfr
- Plot the statewise PSD
- Plot the statewise coherence and PDC
- Plot the fractional occupancy of states (over a tfr)
- 